In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\Barytes\Desktop\NBS_project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.32332202513299674, 'Fn': 1.184383282690447, 'kn': 0.6732799162883472, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 0.6374404631644437, 'Fn': 1.7637226511189565, 'kn': 0.6651454029151388, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.395711266542491, 'Fn': 2.1321345406869066, 'kn': 0.19255662205025992, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.16187321635232887, 'Fn': 3.8199302863134226, 'kn': 0.3183833952999494, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 0.6413215489755134, 'Fn': 3.913825949541608, 'kn': 0.4233228768151258, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.49308149762923636, 'Fn': 2.4920855793255217, 'kn': 0.5819339189335254, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.17404615893711323, 'Fn': 1.5048957788256199, 'kn': 0.3264336993988291, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.350

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
Uniform p: [0.52570672 0.43845893 0.4336311  0.99628099 0.83625805 0.51971064
 0.58889395 0.57981012 0.7173343  0.77667014]
Uniform Dmax: 2.3936391851929404
Uniform r: [23.7649475 23.7649475 23.7649475 23.7649475 23.7649475 23.7649475
 23.7649475 23.7649475 23.7649475 23.7649475]
social welfare: 237.64947497802328
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.6605873227631235', 'MD 2 Dn: 2.0898982619957733', 'MD 3 Dn: 2.39363918519296', 'MD 4 Dn: 0.1449975698161813', 'MD 5 Dn: 0.1959593245661619', 'MD 6 Dn: 0.6941888514389128', 'MD 7 Dn: 0.4375430119900058', 'MD 8 Dn: 0.4548886353427264', 'MD 9 Dn: 0.2641730041134005', 'MD 10 Dn: 0.22985694746670735']
Dmax: 2.3936391851929404  D0: 3
max Dn: 2.39363918519296
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 237.64947497802336 Q_star: 241.75408456829268
rn>=Ln: True
sum Ln: 4.104609590269401


In [4]:
lam_pro, p_pro, r_pro, Dmax_pro = baseline.proportional_baseline(esp,MDs,43)
print(f"Uniform λ: {lam_pro}")
print(f"Uniform p: {p_pro}")  
print(f"Uniform Dmax: {Dmax_pro}")
print(f"Uniform r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Uniform λ: [2.09025991 3.11270752 3.76289958 6.74160743 6.90731927 4.39815949
 2.65591668 5.8750097  6.85821515 7.59790528]
Uniform p: [0.8400625112602841, 0.34090587831472074, 0.36397107757518515, 3.2967540152184913, 2.5371703641888748, 0.8536289147195153, 1.1907565012107326, 1.2435702010724685, 1.965798290302121, 2.290152720278999]
Uniform Dmax: 1.2859583266526118
Uniform r: [ 9.62393296 14.33146583 17.32506713 31.03957436 31.80254154 20.24991816
 12.22831857 27.0496024  31.57645732 34.98212386]
social welfare: 230.20900211930916
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.14308407152681338', 'MD 2 Dn: 0.8803702081765077', 'MD 3 Dn: 1.2859583266526118', 'MD 4 Dn: 0.03984520611344912', 'MD 5 Dn: 0.05289231865318548', 'MD 6 Dn: 0.18534436514671376', 'MD 7 Dn: 0.0958008585011212', 'MD 8 Dn: 0.11339155745548526', 'MD 9 Dn: 0.06852461299682983', 'MD 10 Dn: 0.06502289420130582']
Dmax: 1.2859583266526118  D0: 3
max Dn: 1.2859583266526118
Dmax == max Dn: True
Dn<=Dmax: True
Dmax<=D0: True
pn<

In [7]:
lam_non, p_non, r_non, Dmax_non = baseline.non_cooperative_baseline(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -245.40286262771158
            Iterations: 61
            Function evaluations: 1343
            Gradient evaluations: 61
Non-cooperative λ: [0.00000000e+00 2.56406904e-01 7.79001071e+00 1.93402086e+01
 3.73601117e+00 2.50934706e+00 9.19819027e+00 7.20349174e-15
 7.16982534e+00 0.00000000e+00]
Non-cooperative p: [0, 0.12454906707641597, 0.727936286709583, 1.6513989040880168, 0.4033009472250074, 0.3070166661720434, 0.8404723634602973, 0, 0.6779367949023778, 0]
Non-cooperative Dmax: 0.7787179813619909
Non-cooperative r: [0.07220915 0.07923241 0.29908593 0.69418366 0.1844805  0.17178428
 0.39061186 0.08050853 0.29507086 0.07901784]
social welfare: 245.63459814906685
sum lambda: 50.000000000000014  lambda0: 50
['MD 1 Dn: 0', 'MD 2 Dn: 0.778717981361991', 'MD 3 Dn: 0.778717981361991', 'MD 4 Dn: 0.7787179813619917', 'MD 5 Dn: 0.7787179813619916', 'MD 6 Dn: 0.7787179813619907', 'M

d:\Program\Anaconda\envs\tensorflow\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [6]:
lam_con, p_con, r_con, Dmax_con = baseline.contract_baseline(esp,MDs)
print(f"Contract λ: {lam_con}")
print(f"Contract p: {p_con}")  
print(f"Contract Dmax: {Dmax_con}")
print(f"Contract r: {r_con}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_con, p_con))
print("sum lambda:", np.sum(lam_con), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_con[i],lam_con[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_con, " D0:", esp.D0)
max_Dn = max(md.delay(p_con[i], lam_con[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_con == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_con[i], lam_con[i]) <= Dmax_con for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_con <= esp.D0)
print("pn<=Fn:", all(p_con[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_con[i] <= p_con[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_con), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_con[i] >= md.Ln(p_con[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_con[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -244.72838498087344
            Iterations: 89
            Function evaluations: 2768
            Gradient evaluations: 85
Contract λ: [1.00000000e-06 1.93346889e-02 8.01062707e+00 2.50041139e+01
 9.40233479e-01 1.00000001e-06 8.01203850e+00 1.00000000e-06
 8.01364840e+00 1.00000000e-06]
Contract p: [0.09169906 0.09175389 0.73158227 2.09064575 0.16571401 0.09188076
 0.7316064  0.09022934 0.73153516 0.09099855]
Contract Dmax: 0.896605154800742
Contract r: [0.07044912 0.07048724 0.39604497 1.2583493  0.09725603 0.07055093
 0.39606148 0.069348   0.39600978 0.06994862]
social welfare: 245.2741564300882
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0', 'MD 2 Dn: 0.8966051548008795', 'MD 3 Dn: 0.8966051548008942', 'MD 4 Dn: 0.8966051548008795', 'MD 5 Dn: 0.8966051548009136', 'MD 6 Dn: 0', 'MD 7 Dn: 0.8966051548008684', 'MD 8 Dn: 0', 'MD 9 Dn: 0.8966051548008801', 'MD 10 Dn: 0']
Dmax: 0

In [7]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -243.4064349220435
            Iterations: 60
            Function evaluations: 1336
            Gradient evaluations: 60
SWM λ: [2.95757788 2.46662186 6.84458992 9.8414872  4.62313873 4.06248549
 7.37773116 2.56009636 6.52995507 2.73631633]
SWM p: [0.34014726 0.29897271 0.64987718 0.88910101 0.47186013 0.4287716
 0.6924156  0.30647879 0.62433931 0.32155411]
SWM Dmax: 0.796826935565435
Optimization terminated successfully    (Exit mode 0)
            Current function value: -37.59035133973936
            Iterations: 12
            Function evaluations: 133
            Gradient evaluations: 12
SWM r: [20.69583471 20.65648755 20.67534981 20.68191024 20.66711523 20.69767577
 20.72171966 20.68979103 20.68757418 20.69704961]
social welfare: 245.1600742342796
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.7968259353968623', 'MD 2 Dn: 0.7968259352948482', 'MD 3 Dn: 0.7968259355362222',

In [8]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -35.98812131829895
            Iterations: 59
            Function evaluations: 1889
            Gradient evaluations: 59
Optimal λ: [4.8404622  4.74770872 5.17328246 5.31423996 4.98064244 4.93263002
 5.19162106 4.82944036 5.14614309 4.8438297 ]
Optimal p: [0.48926574 0.48000194 0.5146958  0.52545955 0.49899226 0.49686417
 0.51605325 0.48656778 0.51216807 0.48866867]
Optimal r: [20.81112359 20.82058158 20.56062763 20.51114944 20.65560883 20.72438297
 20.58857432 20.84049204 20.58283629 20.83146823]
Optimal Dmax: 0.8082780146622826
social welfare: 244.7511248467392
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.8082780146962282', 'MD 2 Dn: 0.8082780146168773', 'MD 3 Dn: 0.8082780146737935', 'MD 4 Dn: 0.8082780146416569', 'MD 5 Dn: 0.8082780146600207', 'MD 6 Dn: 0.8082780146613974', 'MD 7 Dn: 0.8082780146635227', 'MD 8 Dn: 0.8082780146686164', 'MD 9 Dn: 0.8082780146749876', 'MD 10

In [9]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

MD SLSQP failed: Positive directional derivative for linesearch
MD SLSQP failed: Iteration limit reached
ADMM λ: [3.74628718 3.29146651 6.23098394 7.73523628 4.79545418 4.43954606
 6.49441783 3.56340588 6.02877314 3.674429  ]
ADMM p: [0.40261112 0.36434955 0.6001704  0.71998901 0.4850307  0.45830063
 0.62113362 0.38613336 0.5836309  0.3959812 ]
ADMM Dmax: 0.801579368372162
ADMM r: [20.74343657 20.71028953 20.63148896 20.60008384 20.66552928 20.71092217
 20.66701551 20.75617656 20.64832051 20.75716806]
social welfare: 245.01122717728902
sum lambda: 50.00000000000001  lambda0: 50
['MD 1 Dn: 0.8015777090872077', 'MD 2 Dn: 0.8015836929447394', 'MD 3 Dn: 0.801580234953847', 'MD 4 Dn: 0.801580030679327', 'MD 5 Dn: 0.8015800689017089', 'MD 6 Dn: 0.801578448712355', 'MD 7 Dn: 0.8015800786464072', 'MD 8 Dn: 0.8015768425568874', 'MD 9 Dn: 0.8015800305253504', 'MD 10 Dn: 0.8015765980458991']
Dmax: 0.801579368372162  D0: 3
max Dn: 0.8015836929447394
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: 

In [10]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("Nash Product Contract:",model.nash_product_log(esp, MDs, lam_con, p_con, r_con))
print("=============================")
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("SW Contract:",model.social_welfare(esp, MDs, lam_con, p_con))
print("=============================")
print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Non-cooperative:", Dmax_non)
print("Dmax Contract:", Dmax_con)

Nash Product Opt: 37.570317091183014
Nash Product SWM: 37.590350900664916
Nash Product ADMM: 37.58306298016239
Nash Product Proportional: 35.455669639833715
Nash Product Uniform: 36.22236437448023
Nash Product Non-cooperative: nan
Nash Product Contract: nan
SW Opt: 244.7511248467392
SW SWM: 245.1600742342796
SW ADMM: 245.01122717728902
SW Proportional: 230.20900211930916
SW Uniform: 230.4668976068324
SW Non-cooperative: 245.63459814906685
SW Contract: 245.2741564300882
Dmax Opt: 0.8082780146622826
Dmax SWM: 0.796826935565435
Dmax ADMM: 0.801579368372162
Dmax Proportional: 1.2859583266526118
Dmax Uniform: 1.3272929506848081
Dmax Non-cooperative: 0.7787179813619909
Dmax Contract: 0.896605154800742


c:\Users\Barytes\Desktop\NBS_project\src\model.py:12: RuntimeWarning: invalid value encountered in log
  ns = [md.omega_n*np.log(rn-md.Ln(pn)) for (md,rn,pn) in zip(MDs,r,p)]
